In [ ]:
! pip install -U tokenizers
!pip install 
import torch
import tokenizers
import os

     |████████████████████████████████| 3.3MB 27.9MB/s 
     |████████████████████████████████| 2.5MB 31.3MB/s 
     |████████████████████████████████| 901kB 41.3MB/s 


In [ ]:
!cat /var/log/colab-jupyter.log

cat: /var/log/colab-jupyter.log: No such file or directory


In [ ]:
#config tokenizer
bwpt = tokenizers.BertWordPieceTokenizer(
    clean_text=True,
    handle_chinese_chars=True,
    strip_accents=True,
    lowercase=True,
)

In [ ]:
print(bwpt)

Tokenizer(vocabulary_size=0, model=BertWordPiece, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], pad_token=[PAD], mask_token=[MASK], clean_text=True, handle_chinese_chars=True, strip_accents=True, lowercase=True, wordpieces_prefix=##)


In [ ]:
#train tokenizer
bwpt.train(
    files=["/content/drive/MyDrive/tweets_txt_132_nostopword.txt"], #korpus
    
    vocab_size=150000,
    min_frequency=2,
    show_progress=True,
    special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],
    limit_alphabet=1000,
    wordpieces_prefix="##",
)

In [ ]:
#save tokenizer
bwpt.save_model("/content/drive/MyDrive/wordpiecev3")

['/content/drive/MyDrive/wordpiecev3/vocab.txt']

In [ ]:
import json
import panda as pd
import sys
f = open("July-6.json")
data - json.load(f)
tweetIds=[]

for key in data["globalObjects"]["tweets"].keys():
  tweetIds.append(key)
df = pd.DataFrame(column = ['tweetId', 'tweet','date'])
for tweetId in tweetIds:
  temp_col = ['tweetId','tweet','date']
  temp_data = data['globalObjects']['tweets']['full_text']
  row = pd.DataFrame([[tweetId,temp_data,'01-07-2020']],columns=temp_col)
  df = df.append(row,ignore_index=True)

df.to_csv('july1.csv',mode='a',hader=False)


**TRAIN MODEL DISTILBERT**



In [ ]:
#distilbert Configuration
from transformers import DistilBertConfig

config = DistilBertConfig(
    vocab_size=150_000,
    max_position_embeddings=512,
    n_layers=7,
    hidden_dim=256,
    
    n_heads=12,
    dropout=0.25,
    activation= "relu"
    
)

ModuleNotFoundError: ignored

In [ ]:
#load pretrained tokenizer
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained("/content/drive/MyDrive/wordpiecev2")

In [ ]:
#configure model
from transformers import DistilBertForMaskedLM
model = DistilBertForMaskedLM(config=config)

In [ ]:
%%time
from transformers import LineByLineTextDataset
#LineByLineTextDataset, reads each line separately, tokenizes and truncates the lines to block_size. Adds special tokens.

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="/content/drive/MyDrive/tweets_txt_132_nostopword.txt",
    block_size=256,
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:124: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:

#data collator,  object that will make batch using dataset as input
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.3
)

In [ ]:
#train
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./model_outputv2",
    overwrite_output_dir=True,
    num_train_epochs=50,
    per_device_train_batch_size=8,
    save_steps=5_000,
    save_total_limit=2,
    prediction_loss_only=True,
    learning_rate=1e-5,
    do_eval=True,
    max_steps=10000,
    logging_steps=100
)
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)
#5e-5

In [ ]:
%%time
trainer.train()

In [ ]:
#simpan
trainer.save_model("/content/drive/MyDrive/model_outputv10")